In [66]:
import requests
import os
import json
import random 
import time
from datetime import datetime
import psycopg2
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [155]:
# Database connection parameters
dbname = "colruyt_tracker"
user = "thibodebras"
password = "postgres"
host = "localhost"  # or the IP address of your PostgreSQL server
port = "5432"

In [229]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

query = "SELECT * FROM raw_data_2024_02_22"

# Use read_sql_query to execute the query and assign the result to a DataFrame
df_raw = pd.read_sql_query(query, conn)

# Don't forget to close the connection when done
conn.close()

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_7723/1599586482.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql_query(query, conn)


In [230]:
df_raw.head()

,run_date,shop_id,page,products
0,2024-02-22,459,1,"[{'productId': '197525', 'technicalArticleNumb..."
1,2024-02-22,459,2,"[{'productId': '6951', 'technicalArticleNumber..."
2,2024-02-22,459,3,"[{'productId': '179589', 'technicalArticleNumb..."
3,2024-02-22,459,4,"[{'productId': '189612', 'technicalArticleNumb..."
4,2024-02-22,459,5,"[{'productId': '333989', 'technicalArticleNumb..."


In [231]:
exploded_df = df_raw.explode('products')

products_normalized = pd.json_normalize(exploded_df['products'].apply(lambda x: x if isinstance(x, dict) else {}))

# Drop the 'products' column from exploded_df since it's no longer needed in its original form
exploded_df.drop(columns=['products'], inplace=True)

# Reset the index of exploded_df to merge it with products_normalized
exploded_df.reset_index(drop=True, inplace=True)

# Concatenate the normalized products DataFrame with the exploded DataFrame
# It's important to reset the index on products_normalized if you haven't already, to ensure alignment
result_df = pd.concat([exploded_df, products_normalized.reset_index(drop=True)], axis=1)
result_df

,run_date,shop_id,page,productId,technicalArticleNumber,commercialArticleNumber,name,description,brand,seoBrand,thumbNail,fullImage,squareImage,content,walkRouteSequenceNumber,isAvailable,isPriceAvailable,inPromo,gtin,topCategoryName,topCategoryId,businessDomain,IsPrivateLabel,IsBiffe,WeightconversionFactor,IsWeightArticle,IsBio,CountryOfOrigin,IsExclusivelySoldInLuxembourg,OrderUnit,ShortName,InSeason,IsNew,LongName,RecentQuanityOfStockUnits,AlcoholVolume,StartSeasonDate,FicCode,price.basicPrice,price.recommendedQuantity,price.unit,price.quantityPrice,price.quantityPriceQuantity,price.measurementUnitPrice,price.measurementUnitQuantityPrice,price.measurementUnit,price.isRedPrice,price.pricePerUOM,price.activationDate,price.recordSource,price.isPromoActive,price.priceChangeCode,price.quantityPricePerUOM,price.quantityActivationDate,price.quantityPriceChangeCode,nutriscoreLabel,EcoscoreLabel,EcoscoreValue,promotion,price.redPriceReason,DepositValue,price.referencePrice,price.retentionPeriod,EndSeasonDate,BiffeDate
0,2024-02-22,459,1,197525,3440812,29018,bananes,"*Origine: Guatemala, Costa Rica\n*Classe 1\n*V...",PAPILLON,Papillon,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,±1kg,9359.0,True,True,False,"[95400141009135, 05425016492581, 0542501649353...",Légumes et fruits,1675,RETAIL_BE,False,False,1000,True,False,GUATEMALA,False,P,BANANES PAPILLON P,True,False,PAPILLON bananes ±1kg,1,0,26/02/2016,E,2.40,1.2,K,2.2,3.4,2.400,2.2,K,False,2.400,02-11-2023,Offline,N,H,2.2,02-11-2023,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-02-22,459,1,969,29628,29628,carottes,"*Origine: Belgique, Pays-Bas\n*Classe 1",NaN,NaN,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,NaN,9856.0,True,True,False,"[05413568000057, 95400141241962, 0541080010006...",Légumes et fruits,1675,RETAIL_BE,False,False,100,True,False,BELGIQUE,False,"P,Kg",CAROTTES P,NaN,False,carottes,"1,1,1",0,NaN,N,0.99,1.0,K,NaN,NaN,0.990,NaN,K,False,0.990,31-01-2024,Offline,N,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-22,459,1,734910,4495794,19139,mandarines,*Origine: Espagne,BONI,Boni Selection,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,NaN,9581.0,True,True,False,"[95400141491329, 95400141896742, 9540014179670...",Légumes et fruits,1675,RETAIL_BE,True,False,100,True,False,ESPAGNE,False,"P,Kg",BONI MANDARINES W,True,False,BONI mandarines,"1,1,1,1",NaN,11/10/2023,I,2.99,1.0,K,NaN,NaN,2.990,NaN,K,False,2.990,20-02-2024,Offline,N,H,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-02-22,459,1,1027,29707,29707,courgettes,*Origine: Espagne\n*Classe 1,NaN,NaN,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,NaN,9746.0,True,True,False,"[08436540550064, 08437011296504, 0542501649047...",Légumes et fruits,1675,RETAIL_BE,False,False,300,True,False,ESPAGNE,False,"P,Kg",COURGETTES P,NaN,False,courgettes,"1,1,1",0,NaN,N,2.39,1.0,K,2.2,3.0,2.390,2.2,K,False,2.390,21-02-2024,Online,N,E,2.2,21-02-2024,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-02-22,459,1,843058,4688137,30635,élevées au sol M,NaN,EVERYDAY,Everyday,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,12pc,8895.0,True,True,False,"[75400141530949, 05400141530940, 1540014153094...",Crémerie,65,RETAIL_BE,True,False,0,False,False,BELGIQUE,False,P,EVD SCHARRELEI M 12ST,NaN,False,EVERYDAY élevées au sol M 12pc,"15,1,15,15,15,15",0,NaN,I,2.22,1.0,NaN,NaN,NaN,0.185,NaN,S,False,0.185,17-01-2024,Offline,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,.

In [232]:
result_df = result_df.drop_duplicates(subset=["shop_id", "productId"])

In [233]:
54572-54364

208

In [234]:
result_df.describe()


,page,walkRouteSequenceNumber,price.basicPrice,price.quantityPrice,price.measurementUnitPrice,price.measurementUnitQuantityPrice,price.pricePerUOM,price.quantityPricePerUOM,price.referencePrice,price.retentionPeriod
count,54557.000000,45604.000000,46395.000000,6287.000000,46385.000000,6287.000000,46385.000000,6287.000000,305.000000,305.0
mean,27.877596,8290.670446,6.856006,5.304462,18.856974,10.548906,18.856974,10.548906,14.331902,30.0
std,15.801252,4869.218138,7.717417,5.145991,63.851560,16.042376,63.851560,16.042376,12.934153,0.0
min,1.000000,1.000000,0.000000,0.275000,0.002000,0.004000,0.002000,0.004000,0.990000,30.0
25%,14.000000,3916.000000,2.590000,2.100000,3.920000,3.450000,3.920000,3.450000,4.290000,30.0
50%,28.000000,8602.500000,4.290000,3.640000,9.180000,7.580000,9.180000,7.580000,9.990000,30.0
75%,42.000000,12240.250000,7.990000,6.660000,17.560000,13.300000,17.560000,13.300000,20.900000,30.0
max,55.000000,18450.000000,199.000000,38.350000,2055.000000,781.670000,2055.000000,781.670000,67.990000,30.0


In [235]:
# result_df.info()

In [236]:
# result_df.loc[result_df['shop_id'] == '459']

In [237]:
# result_df.groupby(result_df["productId"]).count().sort_values("run_date")

In [238]:
# result_df.loc[result_df["productId"]=="192790"]

In [239]:
# grouped = result_df.groupby(["productId"]).count()
# grouped.loc[grouped["run_date"]>1]

In [240]:
# result = result_df.groupby('productId')['price.quantityActivationDate'].nunique()
# result.sort_values()

In [241]:
# result_df.loc[result_df['productId'] == '10']

In [242]:
# result_df.columns.to_list

In [243]:
products_df = result_df[['productId', 'name', 'topCategoryName', 'brand']].drop_duplicates()
products_df.columns = ['ProductID', 'ProductName', 'Category', 'Brand']

# Creating the Prices DataFrame
prices_df = result_df[['productId', 'shop_id', 'price.basicPrice', 'run_date']]
prices_df.columns = ['ProductID', 'ShopID', 'Price', 'Date']

# Creating a simple Shops DataFrame
shops_df = pd.DataFrame(result_df['shop_id'].unique(), columns=['ShopID'])
# Placeholder columns for Shop details (to be updated/filled elsewhere)
shops_df['ShopName'] = np.nan
shops_df['Address'] = np.nan
shops_df['City'] = np.nan

# Resetting indexes
products_df.reset_index(drop=True, inplace=True)
prices_df.reset_index(drop=True, inplace=True)
shops_df.reset_index(drop=True, inplace=True)

# products_df = products_df.set_index(["ProductID"], drop=True)
prices_df['Combined'] = prices_df['ProductID'].astype(str) + '-' + prices_df['ShopID'].astype(str) + '-' + prices_df['Date'].astype(str)


# Step 2: Hash the combined column
# We're using the built-in hash function for simplicity. For consistent results across sessions,
# consider using a stable hash function like hashlib or any other stable hashing mechanism.
prices_df['PriceID'] = prices_df['Combined'].apply(lambda x: hash(x))

# Step 3: Set the hash column as the new index
# prices_df = prices_df.set_index('PriceID')

# Dropping the Combined column as it's no longer needed, unless you want to keep it for reference
prices_df.drop('Combined', axis=1, inplace=True)
# shops_df = shops_df.set_index(["ShopID"], drop=True)

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_7723/2590782205.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['Combined'] = prices_df['ProductID'].astype(str) + '-' + prices_df['ShopID'].astype(str) + '-' + prices_df['Date'].astype(str)
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_7723/2590782205.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['PriceID'] = prices_df['Combined'].apply(lambda x: hash(x))
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/

In [244]:
products_df.head()

,ProductID,ProductName,Category,Brand
0,197525,bananes,Légumes et fruits,PAPILLON
1,969,carottes,Légumes et fruits,NaN
2,734910,mandarines,Légumes et fruits,BONI
3,1027,courgettes,Légumes et fruits,NaN
4,843058,élevées au sol M,Crémerie,EVERYDAY


In [245]:
prices_df.head()

,ProductID,ShopID,Price,Date,PriceID
0,197525,459,2.40,2024-02-22,7064931226952085575
1,969,459,0.99,2024-02-22,4281265726115253933
2,734910,459,2.99,2024-02-22,-5190617032094424398
3,1027,459,2.39,2024-02-22,-2576068116600906229
4,843058,459,2.22,2024-02-22,8548227243563242326


In [246]:
shops_df

,ShopID,ShopName,Address,City
0,459,NaN,NaN,NaN
1,1825,NaN,NaN,NaN
2,471,NaN,NaN,NaN
3,1671,NaN,NaN,NaN


In [247]:
# from sqlalchemy import create_engine

# # PostgreSQL connection string format
# connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"
# engine = create_engine(connection_string)

In [248]:
# Write Products DataFrame to PostgreSQL
# products_df.to_sql('product_test', con=engine, if_exists='replace', index=True)

# # Write Prices DataFrame to PostgreSQL
# prices_df.to_sql('price_test', con=engine, if_exists='replace', index=True)

# # Write Shops DataFrame to PostgreSQL
# shops_df.to_sql('shop_test', con=engine, if_exists='replace', index=True)

In [249]:
# engine.dispose()

In [250]:
def convert_row_to_tuple(row):
    return tuple(row.astype(object).where(pd.notnull(row), None).apply(lambda x: x.item() if isinstance(x, np.generic) else x))

# Adjusted function for writing a DataFrame to PostgreSQL, including type conversion
def write_dataframe_to_postgresql(df, table_name, conn):
    # Convert DataFrame columns to native Python types to avoid numpy.int64 issue
    df = df.map(lambda x: x.item() if isinstance(x, np.generic) else x)
    
    # Prepare data for insertion
    records = [convert_row_to_tuple(row) for index, row in df.iterrows()]
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"
    
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, records)
        conn.commit()
    except Exception as e:
        print(f"Error inserting into {table_name}: {e}")
        conn.rollback()
    finally:
        cursor.close()



In [223]:

# def write_dataframe_to_postgresql(df, table_name, conn):
#     # Convert the dataframe to a list of tuples
#     records = list(df.to_records(index=True))

#     # Create a list of placeholders for each record
#     placeholders = ', '.join(['%s'] * len(df.columns))

#     # Create the INSERT INTO statement
#     columns = ', '.join(df.columns)
#     sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

#     # Create a cursor and execute the SQL statement
#     cursor = conn.cursor()
#     try:
#         cursor.executemany(sql, records)
#         conn.commit()  # Commit the transaction
#     except Exception as e:
#         conn.rollback()  # Rollback the transaction on error
#         print(f"Error inserting into {table_name}: {e}")
#     finally:
#         cursor.close()


In [252]:

# Example usage
write_dataframe_to_postgresql(products_df, 'product_test_2', conn)
print("1")

write_dataframe_to_postgresql(shops_df, 'shop_test_2', conn)
print("2")
write_dataframe_to_postgresql(prices_df, 'price_test_2', conn)
print("3")

1
2
3


In [174]:
def create_tables(conn):
    # SQL statements for creating the tables
    commands = (
        """
        CREATE TABLE IF NOT EXISTS product_test_2 (
            ProductID INT PRIMARY KEY,
            ProductName VARCHAR(255),
            Category VARCHAR(255),
            Brand VARCHAR(255)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS shop_test_2 (
            ShopID INT PRIMARY KEY,
            ShopName VARCHAR(255),
            Address TEXT,
            City VARCHAR(255)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS price_test_2 (
            PriceID VARCHAR(32) PRIMARY KEY,
            ProductID INT,
            ShopID INT,
            Price DECIMAL(10, 2),
            Date DATE,
            FOREIGN KEY (ProductID) REFERENCES product_test_2(ProductID),
            FOREIGN KEY (ShopID) REFERENCES shop_test_2(ShopID)
        );
        """
    )
    cur = conn.cursor()
    try:
        # Execute each command
        for command in commands:
            cur.execute(command)
        cur.close()
        conn.commit()  # Commit the changes to the database
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()  # Rollback on error
    finally:
        if cur is not None:
            cur.close()


In [251]:

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)


In [ ]:

# Call the function to create the tables
create_tables(conn)